In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
!pip install torch
!pip install numpy
!pip install pronouncing
!pip install frozendict
!pip install sacremoses
!pip install Phyme

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 23.9 MB/s 
     |████████████████████████████████| 101 kB 14.0 MB/s 
     |████████████████████████████████| 596 kB 66.2 MB/s 
     |████████████████████████████████| 6.6 MB 50.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 939 kB 40.4 MB/s 
  Created wheel for pronouncing: filename=pronouncing-0.2.0-py2.py3-none-any.whl size=6252 sha256=de283bd5f948630c0c921a45cfdba0ce79a2ed4f8f424cdb4a7ecb2ef14117b3
  

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
%cd drive/MyDrive/eth_courses/Autumn22/nlp/spanningtrees/

/content/drive/MyDrive/eth_courses/Autumn22/nlp/spanningtrees


In [3]:
from poem_generation import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
100%|██████████| 267735/267735 [00:01<00:00, 252809.64it/s]


In [4]:
model = model.to(torch.device('cuda'))
device = torch.device('cuda')

In [ ]:
get_rhyming_words("Lonely")

['extraterritoriality']

In [ ]:
word = "Lonely"
word = re.sub('[^a-z\']', '', word.lower())
syllable_to_words = ph.get_perfect_rhymes(word, num_syllables=1)
output= []
for word_list in syllable_to_words.values():
    output += word_list
print(output[:5])

['be(1)', 'sui(1)', 'the(2)', 'doggie', 'ferry']


In [12]:
def get_rhyming_words(word):
    word = re.sub('[^a-z\']', '', word.lower())
    try:
        syllable_to_words = ph.get_perfect_rhymes(word, num_syllables=1)  # most permissive - only rhyme last syllable
    except KeyError:
        return []
    output = []
    for word_list in syllable_to_words.values():
        output += word_list
    return output
class syllable_line_rhyming_logits(syllable_line_logits):
    def __init__(self, num_syllables=7, sd=3.5):
        super().__init__(num_syllables=num_syllables)
        self.sd = sd
    def __call__(self, input_ids, scores):

        banned_tokens = []
        for beam_index, (beam_input_ids, beam_scores) in enumerate(zip(input_ids, scores)):
            text = tokenizer.decode(beam_input_ids)
            word_syllables = get_text_syllables(text)

            last_rhyme_word_index = get_last_syllable_breakpoint(word_syllables, self.num_syllables)
            if last_rhyme_word_index:
                last_rhyme_word = word_syllables[last_rhyme_word_index][0]
                rhyming_words = set(get_rhyming_words(last_rhyme_word))
                rhyming_words_idxs = np.array([vocab['sym2idx'][word] for word in rhyming_words if
                                               word in vocab['sym2idx']])
            else:
                rhyming_words = set([])
                rhyming_words_idxs = np.array([])

            num_syllables = get_text_num_syllables(text)
            # print(num_syllables)
            num_syllables = num_syllables % self.num_syllables

            to_ban = [idxs for i, idxs in enumerate(self.syllable_idxs) if i + 1 + num_syllables > self.num_syllables]

            just_fit_idxs = self.syllable_idxs[self.num_syllables - num_syllables - 1]
            just_fit_rhyming_intersection = np.intersect1d(rhyming_words_idxs, just_fit_idxs, assume_unique=True)
            # just_fit_rhyming_intersection = set(just_fit_rhyming_intersection)  # indices within just_fit_idxs
            just_fit_idxs_ban = np.array([idx for idx in just_fit_idxs if not idx in just_fit_rhyming_intersection])
            
            beam_scores[just_fit_rhyming_intersection] += self.sd

            to_ban.append(just_fit_idxs_ban)


            if to_ban:
                print(f'num_syllables: {num_syllables}')
                print(text)
                print(quick_neatify_text(text))
                print(len(to_ban))
            banned_tokens.append(np.concatenate(to_ban) if to_ban else [])

        scores = set_scores_to_inf_for_banned_tokens(scores, banned_tokens)
        return scores

class syllable_line_rhyming_logits_smarter(syllable_line_logits):
    def __init__(self, num_syllables=[3,5], sd=3.5):
        super().__init__(num_syllables=num_syllables)
        self.sd = sd

    def __call__(self, input_ids, scores):

        banned_tokens = []
        for beam_index, (beam_input_ids, beam_scores) in enumerate(zip(input_ids, scores)):
            text = tokenizer.decode(beam_input_ids)
            word_syllables = get_text_syllables(text)

            # how many syllables we're into the current line.
            current_line_num_syllables, actual_line_num, last_word_indicator = \
                get_last_word_indicators([sylls for word, sylls in word_syllables], self.num_syllables)


            last_rhyme_word_index = np.where(last_word_indicator == 1)[0]
            if len(last_rhyme_word_index) > 0:
                last_rhyme_word_index = last_rhyme_word_index[-1]
            else:
                last_rhyme_word_index = None

            if last_rhyme_word_index:
                last_rhyme_word = word_syllables[last_rhyme_word_index][0]
                rhyming_words = set(get_rhyming_words(last_rhyme_word))
                rhyming_words_idxs = np.array([vocab['sym2idx'][word] for word in rhyming_words if
                                               word in vocab['sym2idx']])
            else:
                rhyming_words = set([])
                rhyming_words_idxs = np.array([])

            num_syllables = current_line_num_syllables
            target_syllables = self.num_syllables[actual_line_num % len(self.num_syllables)]
            # we're at the end of a line - don't ban as all welcome: 0, 1, 2, etc. syllable words
            if num_syllables == target_syllables:
                num_syllables = 0
                target_syllables = self.num_syllables[(actual_line_num+1) % len(self.num_syllables)]

            to_ban = [idxs for i, idxs in enumerate(self.syllable_idxs) if i + 1 + num_syllables > target_syllables]

            just_fit_idxs = self.syllable_idxs[target_syllables - num_syllables - 1]
            just_fit_rhyming_intersection = np.intersect1d(rhyming_words_idxs, just_fit_idxs, assume_unique=True)
            # just_fit_rhyming_intersection = set(just_fit_rhyming_intersection)  # indices within just_fit_idxs
            just_fit_idxs_ban = np.array([idx for idx in just_fit_idxs if not idx in just_fit_rhyming_intersection])

            beam_scores[just_fit_rhyming_intersection] += self.sd

            to_ban.append(just_fit_idxs_ban)

            if to_ban:
                print(f'num_syllables: {num_syllables}')
                print(text)
                print(quick_neatify_text(text))
                print(len(to_ban))
            banned_tokens.append(np.concatenate(to_ban) if to_ban else [])

        scores = set_scores_to_inf_for_banned_tokens(scores, banned_tokens)
        return scores

In [17]:
# logits_processor = LogitsProcessorList([long_word_encourager()])
# logits_processor = LogitsProcessorList([syllable_line_logits(num_syllables=6)])
# logits_processor = LogitsProcessorList([syllable_line_rhyming_logits(num_syllables=7)])
logits_processor = LogitsProcessorList([syllable_line_rhyming_logits_smarter(num_syllables=[6,6,7,7,8,8])])
logits_processor = LogitsProcessorList([syllable_line_rhyming_logits_smarter(num_syllables=[6, 7, 7, 6], sd = 4.2)])

# prompt = 'The wind rushed towards the man who lifted the spear from'
# prompt = 'Over hill, over dale,'
# prompt = 'Standing here all lonely, wrapped in my own misery, I thought upon the only'
prompt='Standing here all lonely,'
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids'].to(device)
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=3,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=50,
    do_sample=True,
    top_p=0.92,
    top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6,7,7, 6]))
  print('##########')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
3
num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
3
num_syllables: 0
Standing here all lonely,
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0)]
3
num_syllables: 1
Standing here all lonely, yet
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('yet', 1)]
4
num_syllables: 3
Standing here all lonely, located
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('located', 3)]
6
num_syllables: 2
Standing here all lonely, lonely
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('lonely', 2)]
5
num_syllables: 4
Standing here all lonely, yet regardless
[('Standing', 2), ('here', 1), ('all', 1), ('lonely', 2), (',', 0), ('yet', 1), ('regardless', 3)]
7
num_syllables: 0
Standing here all lonely, located consequently
[('Standing', 2), ('here', 1), ('all'

In [15]:
for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6,7,7, 6]))
  print('##########')

 Standing here all lonely ,
wrapped in my own misery ,
I thought upon the only
man I could possibly
go home . officially
I 'm getting somebody
else out of community .
Would I actually
love you ? ... everybody
thinks [ that ] , what would I do
##########
 Standing here all lonely ,
wrapped in my own misery ,
I thought upon the only
honorable entry . ``
Yes , '' he wrote back . early
on in the morning , vaguely
in his mouth , he finally
found out that she was 13 , 18 , 20
##########
 Standing here all lonely ,
wrapped in my own misery ,
I thought upon the only
thing about which I 'd 'couldn't't ' . FrÃ©dÃ©ric Passy , , 15 , , 10 , Theorie algebraischen 2 , 1 , 2.1. , IX
##########


In [22]:
# logits_processor = LogitsProcessorList([long_word_encourager()])
# logits_processor = LogitsProcessorList([syllable_line_logits(num_syllables=6)])
# logits_processor = LogitsProcessorList([syllable_line_rhyming_logits(num_syllables=7)])
logits_processor = LogitsProcessorList([syllable_line_rhyming_logits_smarter(num_syllables=[6,6,7,7,8,8])])
logits_processor = LogitsProcessorList([syllable_line_rhyming_logits_smarter(num_syllables=[7], sd = 4.2)])

# prompt = 'The wind rushed towards the man who lifted the spear from'
# prompt = 'Over hill, over dale,'
prompt = 'Standing here all lonely, wrapped in my own misery, I thought upon the only'
prompt='Swiftly through the trees she ran,'
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids'].to(device)
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=3,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=50,
    num_beams=7,
    do_sample=True
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6,7,7, 6]))
  print('##########')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


num_syllables: 0
Swiftly through the trees she ran,
[('Swiftly', 2), ('through', 1), ('the', 1), ('trees', 1), ('she', 1), ('ran', 1), (',', 0)]
3
num_syllables: 0
Swiftly through the trees she ran,
[('Swiftly', 2), ('through', 1), ('the', 1), ('trees', 1), ('she', 1), ('ran', 1), (',', 0)]
3
num_syllables: 0
Swiftly through the trees she ran,
[('Swiftly', 2), ('through', 1), ('the', 1), ('trees', 1), ('she', 1), ('ran', 1), (',', 0)]
3
num_syllables: 0
Swiftly through the trees she ran,
[('Swiftly', 2), ('through', 1), ('the', 1), ('trees', 1), ('she', 1), ('ran', 1), (',', 0)]
3
num_syllables: 0
Swiftly through the trees she ran,
[('Swiftly', 2), ('through', 1), ('the', 1), ('trees', 1), ('she', 1), ('ran', 1), (',', 0)]
3
num_syllables: 0
Swiftly through the trees she ran,
[('Swiftly', 2), ('through', 1), ('the', 1), ('trees', 1), ('she', 1), ('ran', 1), (',', 0)]
3
num_syllables: 0
Swiftly through the trees she ran,
[('Swiftly', 2), ('through', 1), ('the', 1), ('trees', 1), ('she',

In [21]:
text = tokenizer.decode(outputs[2], skip_special_tokens=True)
print(text)

Standing here all lonely, lonely and lonely lonely.


In [13]:
count_syllables_cmu('idea')
count_syllables_cmu('kg')


3

0

In [11]:
for output in outputs:
  # print(tokenizer.decode(output, skip_special_tokens=True))
  print(poemify(tokenizer.decode(output, skip_special_tokens=True), [6,7,7, 6]))
  print('##########')

 Standing here all lonely ,
wrapped in my own misery ,
I thought upon the only
thing I saw was the â , `` ( ) .
##########
 Standing here all lonely ,
wrapped in my own misery ,
I thought upon the only
possible peaceful realisation , [ ... ] ... ``
##########
 Standing here all lonely ,
wrapped in my own misery ,
I thought upon the only
thing I did for my , 4.7 kg .
##########
 Standing here all lonely ,
wrapped in my own misery ,
I thought upon the only
idea . had many
friends who played a part .
##########


In [17]:
text= tokenizer.decode(outputs[1], skip_special_tokens=True)

In [22]:
[1] * 4

[1, 1, 1, 1]

In [27]:
# text
words = word_tokenize(text)
word_syll_pairs = [(word, count_syllables_cmu(word)) for word in words]
syllables = [6,7,8, 6, 7, 8]
# word_syll_pairs


def poemify(text, syllables):
    """text: a str. syllables: a list of integers or just one integer."""
    new_text = ""
    syll_current = 0
    words = word_tokenize(text)
    syllables = [syllables] if type(syllables) is not list else syllables
    syllables = syllables * len(words)  # make sure to repeat more than enough lines
    syllables_iterator = iter(syllables)
    syll_target = next(syllables_iterator)
    for word in words:
        word_syll = count_syllables_cmu(word)
        if word_syll > 0:
            if syll_current == syll_target:
                syll_current = word_syll
                new_text += '\n' + word
                syll_target = next(syllables_iterator)
            elif syll_current + word_syll > syll_target:
                syll_current = syll_current + word_syll - syll_target
                new_text += word + '\n'  # This shouldn't really happen.
                syll_target = next(syllables_iterator)
            else:
                syll_current += word_syll
                new_text += ' ' + word
        else:
            new_text += ' ' + word

    return new_text


poem = poemify(text, [6,7,8])
print(poem)


 Over hill , over dale ,
meeting trade football fans , mail ,
and local teams , including `` Catalunya Catalunya '' , `` Catalunya '' ( ale ) ,
formed the youth of the dale .
possible collegiate footballers


In [25]:
text

'Over hill, over dale, meeting trade football fans, mail, and local teams, including "Catalunya Catalunya", "Catalunya" (ale), formed the youth of the dale. possible collegiate footballers'

In [ ]:
a = [1,2,3,4]
a.pop()

4

In [ ]:
text = tokenizer.decode(outputs[0], skip_special_tokens=True)
get_text_syllables(text)

[('Over', 2),
 ('hill', 1),
 (',', 0),
 ('over', 2),
 ('dale', 1),
 (',', 0),
 ('hill', 1),
 ('and', 1),
 ('stream', 1),
 (',', 0),
 ('ravine', 2),
 (',', 0),
 ('dale', 1),
 ('and', 1),
 ('creek', 1),
 ('.', 0),
 ('is', 1),
 ('a', 1),
 ('common', 2),
 ('topographical', 0),
 ('scale', 1),
 ('that', 1),
 ('lives', 1),
 ('over', 2),
 ('it', 1),
 ('with', 1),
 ('watercourses', 0),
 (',', 0),
 ('crags', 0),
 (',', 0),
 ('shallows', 0),
 (',', 0),
 ('riverbanks', 0),
 (',', 0),
 ('hail', 1),
 ('and', 1),
 ('mountain', 2),
 ('summits', 2),
 ('at', 1),
 ('7,100', 0),
 ('feet', 1),
 ('(', 0),
 ('2,200', 0),
 ('metres', 0),
 (')', 0),
 ('.', 0)]

In [ ]:
text = "Standing here all lonely if you have to"
word_syllables = get_text_syllables(text)
self = syllable_line_rhyming_logits(num_syllables=6)


In [ ]:
last_rhyme_word_index = get_last_syllable_breakpoint(word_syllables, self.num_syllables)
if last_rhyme_word_index:
    last_rhyme_word = word_syllables[last_rhyme_word_index][0]
    rhyming_words = set(get_rhyming_words(last_rhyme_word))
    rhyming_words_idxs = np.array([vocab['sym2idx'][word] for word in rhyming_words if
                                    word in vocab['sym2idx']])
else:
    rhyming_words = set([])
    rhyming_words_idxs = np.array([])

In [ ]:
num_syllables = get_text_num_syllables(text)
# print(num_syllables)
num_syllables = num_syllables % self.num_syllables
print(num_syllables)
just_fit_idxs = self.syllable_idxs[self.num_syllables - num_syllables - 1]
just_fit_rhyming_intersection = np.intersect1d(rhyming_words_idxs, just_fit_idxs, assume_unique=True)
just_fit_rhyming_intersection = set(just_fit_rhyming_intersection)  # indices within just_fit_idxs
just_fit_idxs_ban = np.array([idx for idx in just_fit_idxs if not idx in just_fit_rhyming_intersection])

4


In [ ]:
[vocab['idx2sym'][x] for x in just_fit_idxs[:5]]
len(just_fit_idxs),
len(rhyming_words_idxs),
len(just_fit_rhyming_intersection),
len(just_fit_idxs_ban)
[vocab['idx2sym'][x] for x in just_fit_rhyming_intersection][:5]

['also', 'after', 'into', 'other', 'over']

(44382,)

(3620,)

(1098,)

43284

['aptly', 'bungee', 'hoary', 'tutti', 'gangly']

In [ ]:
get_rhyming_words("Lonely")

['extraterritoriality']

In [ ]:
get_rhyming_words("Lonely")

(9, ['extraterritoriality'])

In [ ]:
pronouncing.rhymes('Lonely')

['conely', 'lonely', 'only']

In [ ]:
syllable_to_words = ph.get_perfect_rhymes("Lonely", num_syllables=1)
syllable_to_words

In [ ]:
for word_list in syllable_to_words.values():
  print(word_list[:4])

['be(1)', 'sui(1)', 'the(2)']
['doggie', 'ferry', 'tammy', 'lovely']
['helplessly', 'salary', 'natalie', 'confetti']
['democracy', 'personally', 'secretary', 'elementary']
['negativity', 'anniversary', 'immediately', 'necessarily']
['unconditionally', 'revolutionary', 'interplanetary', 'internationally']
['anaesthesiology', 'anesthesiology', 'assicurazioni', 'cardiopulmonary']
['counterrevolutionary', 'heterosexuality', 'micropaleontology']
['extraterritoriality']


In [ ]:
import Phyme
Phyme.version

'0.0.9'

In [ ]:
from Phyme import Phyme

ph = Phyme()
ph.get_perfect_rhymes('lonely', num_syllables=None)
a = ph.get_perfect_rhymes('lonely', num_syllables=1)
b = ph.get_perfect_rhymes('lonely', num_syllables=2)
a == b

{2: ['lonely', 'conely', 'only']}

False

In [ ]:
len(a[2]),
len(b[2])

(5107,)

3

In [ ]:
foo = ph.get_family_rhymes('"lonely', num_syllables=None)

KeyError: ignored

In [ ]:
word = "can't"
re.sub('[^a-z\']', '', word.lower())

"can't"

In [ ]:
for output in outputs:
  print(tokenizer.decode(output, skip_special_tokens=True))
  print('#######')

The wind rushed towards the man who lifted the spear from the another moon.8.
#######
The wind rushed towards the man who lifted the spear from the furnace venting batch.
#######


In [ ]:
logits_processor = LogitsProcessorList([long_word_encourager()])

prompt = 'The wind rushed towards the man who lifted the spear from'
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids']
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=2,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=20,
    do_sample=True,
    top_p=0.92,
    top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

In [ ]:
import numpy as np
syllable_words = []
idx_to_num_syllables = []
for i in range(15):
  syllable_words.append([])
for word in tqdm.tqdm(vocab['idx2sym'], total=len(vocab['idx2sym'])):
  ns = count_syllables_cmu(word)
  syllable_words[ns].append(word)
  idx_to_num_syllables.append(ns)

idx_to_num_syllables = np.array(idx_to_num_syllables)

100%|██████████| 267735/267735 [00:00<00:00, 282864.67it/s]


In [ ]:
# tokenizerxl = AutoTokenizer.from_pretrained('transfoxl')
# modelxl = AutoModelForCausalLM.from_pretrained("transfoxl")
import transformers
import sacremoses as sm

from transformers import TransfoXLTokenizer, TransfoXLModel, TransfoXLLMHeadModel
import torch

tokenizer = TransfoXLTokenizer.from_pretrained("transfo-xl-wt103")
model = TransfoXLLMHeadModel.from_pretrained("transfo-xl-wt103")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# outputs = model(**inputs)

In [ ]:
class long_word_encourager(LogitsProcessor):
    def __init__(self):
        self.idxs_of_num_syllables = []
        for num_syllables in range(15):
            self.idxs_of_num_syllables.append(np.where(idx_to_num_syllables == num_syllables)[0])

    def __call__(self, input_ids, scores):
        print('hi')
        # return scores
        mean = torch.mean(scores)
        sd = torch.std(scores)
        for beam_index, (beam_input_ids, beam_scores) in enumerate(zip(input_ids, scores)):
            for num_syllables in range(4, 10):
                beam_scores[self.idxs_of_num_syllables[num_syllables]] += sd
        return scores

logits_processor = LogitsProcessorList([long_word_encourager()])

prompt = 'The wind rushed towards the man who lifted the spear from'
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids']
outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=2,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=20,
    do_sample=True,
    top_p=0.92,
    top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


hi
hi


RuntimeError: ignored

In [ ]:
type(model)

transformers.models.transfo_xl.modeling_transfo_xl.TransfoXLLMHeadModel

In [ ]:
for output in outputs:
  print(tokenizer.decode(output, skip_special_tokens=True))
  print('#######')

The wind rushed towards the man who lifted the spear from above the level of parking about 2.35
#######
The wind rushed towards the man who lifted the spear from a box on his wrist. After wearing the
#######


In [ ]:
tokenizer.vocab_files_names

{'pretrained_vocab_file': 'vocab.pkl',
 'pretrained_vocab_file_torch': 'vocab.bin',
 'vocab_file': 'vocab.txt'}

In [ ]:
import pickle
with open('vocab.pkl', 'rb') as file:
  vocab = pickle.load(file)

In [ ]:
vocab['idx2sym'][:5]

['<eos>', 'the', ',', '.', 'of']

In [ ]:
vocab['sym2idx']['directions']

6409

In [ ]:
tokenizer('directions to the mansion, please.')

{'input_ids': [6409, 6, 1, 6233, 2, 12301, 3]}

In [ ]:
transformers.models.transfo_xl.tokenization_transfo_xl.VOCAB_FILES_NAMES

{'pretrained_vocab_file': 'vocab.pkl',
 'pretrained_vocab_file_torch': 'vocab.bin',
 'vocab_file': 'vocab.txt'}

In [ ]:
import tqdm

In [ ]:
def count_syllables_cmu(word):
    lower_word = word.lower()
    if lower_word in cmu:
        return max([len([y for y in x if y[-1] in string.digits])
                    for x in cmu[lower_word]])
    else:  # word isn't in cmu - hopefully is just punctuation
        # if re.search('[a-zA-Z]', lower_word):
        #     return count_syllables_sonhier(word)
        return 0

In [ ]:
import string
import nltk
nltk.download('cmudict')
from nltk.corpus import cmudict
cmu = cmudict.dict()

syllable_words = []
for i in range(15):
  syllable_words.append([])
for word in tqdm.tqdm(vocab['sym2idx'], total=len(vocab['idx2sym'])):
  ns = count_syllables_cmu(word)
  syllable_words[ns].append(word)

[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


True

100%|██████████| 267735/267735 [00:00<00:00, 288314.67it/s]


In [ ]:
count_syllables_cmu('other')

2

In [ ]:
[len(x) for x in syllable_words]

[163805, 17892, 44382, 26153, 11009, 3610, 772, 96, 15, 1, 0, 0, 0, 0, 0]

In [ ]:
!ls /usr/local/lib/python3.7/dist-packages/transformers/models/transfo_xl

configuration_transfo_xl.py
convert_transfo_xl_original_tf_checkpoint_to_pytorch.py
__init__.py
modeling_tf_transfo_xl.py
modeling_tf_transfo_xl_utilities.py
modeling_transfo_xl.py
modeling_transfo_xl_utilities.py
__pycache__
tokenization_transfo_xl.py


In [ ]:
dir(transformers.models.transfo_xl)

['AdaptiveEmbedding',
 'TFAdaptiveEmbedding',
 'TFTransfoXLForSequenceClassification',
 'TFTransfoXLLMHeadModel',
 'TFTransfoXLMainLayer',
 'TFTransfoXLModel',
 'TFTransfoXLPreTrainedModel',
 'TF_TRANSFO_XL_PRETRAINED_MODEL_ARCHIVE_LIST',
 'TRANSFO_XL_PRETRAINED_CONFIG_ARCHIVE_MAP',
 'TRANSFO_XL_PRETRAINED_MODEL_ARCHIVE_LIST',
 'TransfoXLConfig',
 'TransfoXLCorpus',
 'TransfoXLForSequenceClassification',
 'TransfoXLLMHeadModel',
 'TransfoXLModel',
 'TransfoXLPreTrainedModel',
 'TransfoXLTokenizer',
 '__all__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_class_to_module',
 '_import_structure',
 '_modules',
 '_name',
 '_objects',
 'configuration_transfo_xl',
 'load_tf_weights_in_transfo_xl',
 'modeling_tf_transfo_xl',
 'modeling_tf_transfo_xl_utilities',
 'modeling_transfo_xl',
 'modeling_transfo_xl_utilities',
 'tokenization_transfo_xl']

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
tokenizergpt = AutoTokenizer.from_pretrained('gpt2')

{'input_ids': [6409]}

In [ ]:
tokenizer('directions')
tokenizergpt('directions')

{'input_ids': [6409]}

{'input_ids': [12942, 507], 'attention_mask': [1, 1]}

In [ ]:
tokenizer.vocab_size

267735

In [ ]:
tokenizer.vocab_files_names['pretrained_vocab_file']

'vocab.pkl'

In [ ]:
super(type(outputs))

<super: transformers.models.transfo_xl.modeling_transfo_xl.TransfoXLModelOutput,
        None>

In [ ]:
~transformers.utils.is_sacremoses_available()

True

In [ ]:
from beam_search import *

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [ ]:
foo = n_word_lines2()

In [ ]:
len(foo.keys)

50256

In [ ]:
len(foo.add_on_words)

17122

In [ ]:
[len(x) for x in foo.syllable_idxs]

[4117, 2656, 766, 228, 36, 2, 0, 0, 0]

In [ ]:
foo.keys

array(['Ġphyl', 'ĠASAP', 'Ġdehuman', ..., 'ĠSilent', 'Ġgenitals',
       'endered'], dtype='<U128')

In [ ]:
tokenizer2 = AutoTokenizer.from_pretrained('gpt2', add_prefix_space=False)

In [ ]:
foo.keys[0].strip('Ġ ').lower()

'phyl'

In [ ]:
count_syllables_sonhier(foo.keys[0].strip('Ġ ').lower())

1

In [ ]:
syllable_words = []
def cs_son(x):
  x = x.strip('Ġ ').lower()
  x = re.sub(r'[^a-zA-Z]', '', x)
  return count_syllables_sonhier(x)
  
cs_son = np.vectorize(cs_son)
for i in range(1, 10):
    print(i)
    idxs = np.where(cs_son(foo.keys) == i)
    syllable_words.append(foo.keys[idxs[0]])

1
2
3
4
5


KeyboardInterrupt: ignored

In [ ]:
list(tokenizer2.vocab.keys())

['aughters',
 'iological',
 'Ġarche',
 'Ġ376',
 'Ġprofessors',
 'Ġspectrum',
 'ĠApr',
 'istrate',
 'PF',
 'NOW',
 'ĠFact',
 'Ġimposed',
 'ork',
 'Ġtrivia',
 'Take',
 'ĠOg',
 'ĠHir',
 'ĠDefin',
 'ĠTome',
 'ĠActive',
 'ĠFitzgerald',
 'ĠAdvertisement',
 '021',
 'Ġclient',
 '368',
 'Ġprimarily',
 'ĠSiberian',
 'safe',
 'Ġbroken',
 'ampa',
 'ston',
 'ĠEgyptian',
 '555',
 'ĠBerry',
 'Ġsilent',
 'Ġsample',
 '^^',
 'Ġchap',
 'ĠCycling',
 'hig',
 'Ġwaning',
 'ĠSerial',
 'RC',
 'Ġdystop',
 'Ġfeeds',
 'Ġ1906',
 'Ġ[âĢ¦]',
 'MAS',
 'ĠGaul',
 'POL',
 '034',
 'ĠForms',
 'Ġfurthermore',
 'Ġexcluded',
 'Ġpreserved',
 'ĠLia',
 'ĠJay',
 'Ġburying',
 'Ġpat',
 'aturally',
 'Ġeuth',
 'Ġcharg',
 'ĠSolution',
 'section',
 'Present',
 'mit',
 'Ġly',
 'Ġmuscular',
 'ĠAnonymous',
 'Ġchili',
 'Ġfalse',
 'anes',
 'Ġterrestrial',
 'ĠPresents',
 'ĠAuto',
 'OA',
 'Ġ1920',
 'Ġsupreme',
 'Ġrest',
 'threatening',
 'Ġconnectors',
 'ĠPriebus',
 'Ġtuition',
 'Ġalteration',
 'ĠSullivan',
 'Atl',
 'ĠdÃ©',
 'clean',
 '1024',


In [ ]:
from transformers import (
        BeamSearchScorer,
        LogitsProcessorList,
        StoppingCriteriaList,
        MaxLengthCriteria
    )
import torch

# how many beams to track during the Viterbi algorithm
num_beams = 5
# how many beams to return after the algorithm
num_return_beams = 5

# the prompt to continue
prompt = 'My friend Jim'

# tokenizing the prompt
prompt_tokenized = tokenizer(prompt, return_tensors='pt')
prompt_tokenized = prompt_tokenized['input_ids']

# instantiating a BeamSearchScorer
beam_scorer = BeamSearchScorer(
    batch_size=prompt_tokenized.shape[0],
    num_beams=num_beams,
    num_beam_hyps_to_keep=num_return_beams,
    device=model.device
)

# instantiating a list of LogitsProcessor instances
# using our custom ABCLogits class
# logits_processor = LogitsProcessorList([ABCLogits(tokenizer.vocab)])
# logits_processor = LogitsProcessorList([n_word_lines2(n_syllables=7)])
logits_processor = LogitsProcessorList([])

# # printing the output beams
# for index, output_tokenized in enumerate(generated):
#   output = tokenizer.decode(output_tokenized)
#   print(f'beam {index}: {output}')

outputs = model.generate(
    input_ids=prompt_tokenized,
    num_return_sequences=5,
    no_repeat_ngram_size=2,
    remove_invalid_values=True,
    logits_processor=logits_processor,
    max_length=30,
    do_sample=True,
    top_p=0.92,
    top_k=0
    # stopping_criteria=StoppingCriteriaList([MaxLengthCriteria(max_length=20)])
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
for output in outputs:
  print(tokenizer.decode(output, skip_special_tokens=True))
  print('############')

Strong son of god, immortal love,
 who weeps with many bones.


Let us battle these merciless demons!

(radio: Cros
############
Strong son of god, immortal love,
 who weeps to death in glad tears, that his sprightliness hath been assailed, he have
############
Strong son of god, immortal love,
 who we still refuse to accept.

The Lord has sent me. Blind, barren. Sleeping in
############
Strong son of god, immortal love,
 who weeps for parents whose loves were broken, I am your son."

This may be uttered
############
Strong son of god, immortal love,
 who we know as the Aett, leads a godly life.

Deathwards and forwards

############
